In [1]:
import sys
sys.path.append("/s/ls4/users/grartem/RL_robots/CommandClassifier")

In [2]:
import os
import json
import numpy as np
import pandas as pd
from simpletransformers.language_representation import RepresentationModel
from simpletransformers.config.model_args import ModelArgs

In [3]:
ParsedData = []
with open("/s/ls4/users/grartem/RL_robots/CommandClassifier/models/CommandParser_tokenCLS_copy-1_ipynb_data5/checkpoint-116700-epoch-20/predict_test_one_cmd_v4_05072022_low_split.jsonlines", "r") as f:
    for line in f:
        ParsedData.append(json.loads(line))

In [5]:
with open("/s/ls4/users/grartem/RL_robots/CommandClassifier/Data/Interim/labels_names.json", "r") as f:
    attribute_names = json.load(f)
dictionaries_folder = "/s/ls4/users/grartem/RL_robots/CommandClassifier/Data/External/dict/"
attributes_dictionaries = {}
label_to_dictname = dict(zip(["A", "D", "M", "DH", "ON", "N", "RN", "S", "G"],['action', 'direction', 'meters', 'degshours', 'object', 'nearest', 'relation', 'self', 'gaze']))
label_to_attrname = dict(zip(["A", "D", "M", "DH", "ON", "N", "RN", "S", "G"],["action", 'direction', 'meters', 'degshours', 'object1', 'nearest', 'relation1', 'self', 'gaze']))
hours_map = {
    "1hour": "30",
    "2hour": "60",
    "3hour": "90",
    "4hour": "120",
    "5hour": "150",
    "6hour": "180",
    "7hour": "210",
    "8hour": "240",
    "9hour": "270",
    "10hour": "300",
    "11hour": "330",
    "12hour": "360",
    "o1hour": "15",
    "o2hour": "45",
    "o3hour": "75",
    "o4hour": "105",
    "o5hour": "135",
    "o6hour": "165",
    "o7hour": "195",
    "o8hour": "225",
    "o9hour": "255",
    "o10hour": "285",
    "o11hour": "315",
    "o12hour": "345",
}
for phrases_list_file in os.listdir(dictionaries_folder):
    with open(dictionaries_folder + phrases_list_file, "r") as f:
        phrases = f.readlines()
    phrases_list_file = phrases_list_file.split("_")
    phrases = [x.strip() for x in phrases if x.strip()!=""]
    
    label = phrases_list_file[0]
    if label=="O":
        continue
    attr_name = "_".join(phrases_list_file[1:])    
    if attr_name in ["forward", "backward", "right", "left", "north", "south", "east", "west"]:
        attr_name = "dir_" + attr_name
    if "hour" in attr_name:
        attr_name = hours_map[attr_name]
    if attr_name in ["rotate", "move"]:
        attr_name += "_dir"
    if attr_name in ["additional_verb"]:
        continue
    if attr_name in ["focus", "focus_obj"]:
        attr_name = "has_gaze_focus_on"
    if attr_name in ["nearest"]:
        attr_name = "nearest_from_type"
    if attr_name in ["after_turn_around"]: # degshours
        attr_name = "180"
    if attr_name in ["turn_around"]: # action
        attr_name = "rotate_on" # not sure
    
    if attr_name in ["degrees", "meters"]:
        attr_name = attr_name.replace("degrees", "degshours")
        if label_to_dictname[label] not in attributes_dictionaries:
            attributes_dictionaries[label_to_dictname[label]] = [[],[]]
        for val in attribute_names[attr_name]:
            if val=="":
                continue
            for postfix in phrases:
                phrase = val+" " + postfix
        
                attributes_dictionaries[label_to_dictname[label]][0].append(phrase)
                dict_class_id = attribute_names[label_to_attrname[label]].index(val)
                attributes_dictionaries[label_to_dictname[label]][1].append(dict_class_id)
        continue
    if label_to_dictname[label] not in attributes_dictionaries:
        attributes_dictionaries[label_to_dictname[label]] = [[],[]]
    attributes_dictionaries[label_to_dictname[label]][0].extend(phrases)
    dict_class_id = attribute_names[label_to_attrname[label]].index(attr_name)
    attributes_dictionaries[label_to_dictname[label]][1].extend([dict_class_id for x in phrases])
    
    if attr_name in ["rotate_dir", "move_dir"]:
        attr_name = attr_name.replace("_dir", "_to")
        dict_class_id = attribute_names[label_to_attrname[label]].index(attr_name)
        attributes_dictionaries[label_to_dictname[label]][0].extend(phrases)
        attributes_dictionaries[label_to_dictname[label]][1].extend([dict_class_id for x in phrases])
        attr_name = attr_name.replace("_to", "_on")
        dict_class_id = attribute_names[label_to_attrname[label]].index(attr_name)
        attributes_dictionaries[label_to_dictname[label]][0].extend(phrases)
        attributes_dictionaries[label_to_dictname[label]][1].extend([dict_class_id for x in phrases])

In [4]:
# Собираем словари для каждого атрибута по существующим командам
# одинаковые фразы могут относиться к разным классам. тут никуда не денешься
from collections import defaultdict
attributes_dictionaries = defaultdict(list)
attributes_list = ["action", 'direction', 'meters', 'degshours', 'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3', 'self', 'gaze']
labels_list = ["A", "D", "M", "DH", "O1", "N", "R1", "O2", "R2", "O3", "S", "G"]
dictionaries_names_list = ['action', 'direction', 'meters', 'degshours', 'object', 'nearest', 'relation', 'object', 'relation', 'object', 'self', 'gaze']
with open("../../Data/Interim/data_v5_split.jsonl", "r") as f:
    for line in f:
        line = json.loads(line)
        for lab, dictionary_name in zip(labels_list,
                                      dictionaries_names_list):
            phrase = " ".join([x for x,y in zip(line[0], line[1]) if y==lab])
            if phrase!="":
                attributes_dictionaries[dictionary_name].append((phrase, line[2][labels_list.index(lab)]))
for k in attributes_dictionaries.keys():
    attributes_dictionaries[k] = set(attributes_dictionaries[k])
for k in attributes_dictionaries.keys():
    attributes_dictionaries[k] = [[x[0] for x in attributes_dictionaries[k]], [x[1] for x in attributes_dictionaries[k]]]

In [6]:
import random
import torch
class SiameseDataset(torch.utils.data.Dataset):
    def __init__(self, attributes_dictionaries, tokenizer, use_cuda = False):
        '''
        '''
        self.attributes_tuples = []
        for k in attributes_dictionaries.keys():
            for i in range(len(attributes_dictionaries[k][0])):
                phrase, cls  = attributes_dictionaries[k][0][i], attributes_dictionaries[k][1][i]
                self.attributes_tuples.append((k, phrase, cls))
        self.tokenizer = tokenizer
        self.use_cuda=use_cuda
        #if use_cuda:
        #    self.X = self.X.to('cuda') 
        #    self.y = self.y.to('cuda')
        
        
    def __len__(self):
        #total_len = 0
        #for k in self.attributes_dictionaries.keys():
        #    total_len += len(self.attributes_dictionaries[k])
        return len(self.attributes_tuples)

    def __getitem__(self, idx):
        if type(idx)==int:
            idx = torch.tensor(idx)
        anchor = self.attributes_tuples[idx]
        pos_samples = [x for x in self.attributes_tuples if (x[0]==anchor[0]) and (x[2]==anchor[2])]
        neg_samples = [x for x in self.attributes_tuples if (x[0]==anchor[0]) and (x[2]!=anchor[2])]
        pos_sample = random.sample(pos_samples, 1)[0]
        neg_sample = random.sample(neg_samples, 1)[0]
        #не проверял 64, да и вообще это костыль
        sample = self.tokenizer([anchor[1], pos_sample[1], neg_sample[1]], padding=True, pad_to_multiple_of=8,return_tensors='pt')
        if self.use_cuda:
            for k in sample.keys():
                sample[k] = sample[k].cuda()
        sample = {
            "anchor_input_ids": sample['input_ids'][0],
            "pos_input_ids": sample['input_ids'][1],
            "neg_input_ids": sample['input_ids'][2],
            "anchor_token_type_ids": sample['token_type_ids'][0],
            "pos_token_type_ids": sample['token_type_ids'][1],
            "neg_token_type_ids": sample['token_type_ids'][2],
            "anchor_attention_mask": sample['attention_mask'][0],
            "pos_attention_mask": sample['attention_mask'][1],
            "neg_attention_mask": sample['attention_mask'][2]
        }
        return sample

In [7]:
from transformers import BertConfig, BertModel, BertTokenizer

# Download model and configuration from huggingface.co and cache.
tokenizer = BertTokenizer.from_pretrained("/s/ls4/groups/g0126/transformers_models/cointegrated/rubert-tiny2/")
model = BertModel.from_pretrained("/s/ls4/groups/g0126/transformers_models/cointegrated/rubert-tiny2/")
model.to("cuda")
_ = model.train()

Some weights of the model checkpoint at /s/ls4/groups/g0126/transformers_models/cointegrated/rubert-tiny2/ were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
attributes_dictionaries_for_train = {}
for k in attributes_dictionaries.keys():
    if k in ["gaze", "nearest"]:
        continue
    attributes_dictionaries_for_train[k] = attributes_dictionaries[k]
dataset = SiameseDataset(attributes_dictionaries_for_train, tokenizer, use_cuda=True)
trainloader = torch.utils.data.DataLoader(dataset, batch_size=512,
                                          shuffle=True, num_workers=0)

In [9]:
len(dataset)

1028

In [10]:
nonzeros_counts = []
for sample in dataset:
    nonzeros_counts.append(sample["anchor_input_ids"].count_nonzero())
    nonzeros_counts.append(sample["pos_input_ids"].count_nonzero())
    nonzeros_counts.append(sample["neg_input_ids"].count_nonzero())
max(nonzeros_counts), min(nonzeros_counts), sum(nonzeros_counts) / len(nonzeros_counts)

(tensor(7, device='cuda:0'),
 tensor(3, device='cuda:0'),
 tensor(4.3200, device='cuda:0'))

In [11]:
import torch
class SiameseNetwork(torch.nn.Module):
    def __init__(self, learning_rate, model, **kwargs):
        super().__init__()
  
        self.learning_rate = learning_rate
        self._encoder = model
        self.loss = torch.nn.TripletMarginLoss(margin=1.0, p=2)
  
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)
  
    def _loss(self, anchor, pos, neg):
        return self.loss(anchor, pos, neg)
  
    def forward(self, anchor_input_ids, pos_input_ids, neg_input_ids,
               anchor_token_type_ids, pos_token_type_ids, neg_token_type_ids,
               anchor_attention_mask, pos_attention_mask, neg_attention_mask):
        anchor = self._encoder(anchor_input_ids, anchor_token_type_ids, anchor_attention_mask)
        emb1 = self._encoder(pos_input_ids, pos_token_type_ids, pos_attention_mask)
        emb2 = self._encoder(neg_input_ids, neg_token_type_ids, neg_attention_mask)
        return anchor["pooler_output"], emb1["pooler_output"], emb2["pooler_output"]

In [12]:
seamiseNN = SiameseNetwork(learning_rate=0.00001, model=model)
optim = seamiseNN.configure_optimizers()

In [13]:
i=0
for epoch in range(5):
    epoch_stat = {}
    batch_losses = []
    for batch in trainloader:
        #with torch.cuda.amp.autocast():    
        returns = seamiseNN(**batch)
        loss = seamiseNN.loss(*returns)
        loss.backward()
        optim.step()
        
        batch_losses.append(loss)
    epoch_stat["avg_loss"] = sum(batch_losses) / len(batch_losses)
    print(epoch_stat)

{'avg_loss': tensor(0.9637, device='cuda:0', grad_fn=<DivBackward0>)}
{'avg_loss': tensor(0.8792, device='cuda:0', grad_fn=<DivBackward0>)}
{'avg_loss': tensor(0.8832, device='cuda:0', grad_fn=<DivBackward0>)}
{'avg_loss': tensor(0.9876, device='cuda:0', grad_fn=<DivBackward0>)}
{'avg_loss': tensor(0.8277, device='cuda:0', grad_fn=<DivBackward0>)}


In [14]:
seamiseNN._encoder.save_pretrained("/s/ls4/users/grartem/RL_robots/CommandClassifier/models/Prototype_pretrainedNormalizeModel",)